# Анализ данных ритейла. Нахождение точек роста выручки.

Дисклеймер: особенность работы в демонстрации только кода, без расскрытия данных, в виду конфедециального соглашения с компанией.


Работа направлена на создание гипотезы и подготовку данных к проведению A/B теста по ее проверке. Гипотеза появилась в ходе исследования, но здесь представлена вначале для лучшейго понимания процесса.

Гипотеза:                             
Если распределить магазины на сегменты по внешним данным и в каждом сегменте выбрать магазин с растущей выручкой за последние два месяца (топовый), а в магазинах с падающей выручкой (нетоповых), похожих на 97% по ассортименту товаров на магазин с растущей выручкой, и закупать количество каждого товара в нетоповых столько же, сколько в топовых, то это даст рост выручки сети магазинов.

Работа разделена на этапы:\
1). Разделение пользователей на сегманты;\
2). Нахождение магазинов с падающей выручкой относительно предыдущего месяца;\
3). Выбор магазинов в каждом сегменте по ассортименту;\
4). Подготовка к A/B тесту.

# 1). Разделение пользователей на сегменты.

Ниже представлено разделение пользователей на сегменты по уровню спроса. При этом уровень спроса будет определяться четырьмя параметрами: расстояние до метро, расстояние до ближайшей автобусной остановки, средняя дневная выручка, площадь магазина. Каждому магазину будет присвоен рейтинг: от 4 до 12. От 12 до 10 - это магазины сегмента "the best location", от 9 до 7 - это магазины из сегмента "good location", от 6 до 4 - это магазины из сегмента "bad location".

In [1]:
import pandas as pd
import datetime
#считывание данных
shops_infa=pd.read_csv('Outside/zao_info_full.csv', sep=',')
shops_infa.head(40)

FileNotFoundError: [Errno 2] No such file or directory: 'Outside/zao_info_full.csv'

In [2]:
#Удаление ненужных столбцов из датафрейма внешних данных.
shops_infa=shops_infa.drop(['adress','latitude','city_tt','district','longitude','format','shtat',\
                           'qty_kassa','qty_kassa_so','vegetables_count','meat_count'\
                           ,'cafe','coffee','nearest_food_distance','universam_count_500m','city_center_distance','radius_200m_food_count',\
                           'avg_daily_bills_qty','Hours','avg_monthly_bills_qty'],axis=1)
shops_infa

NameError: name 'shops_infa' is not defined

In [3]:
#Ниже представлены квантили для каждого параметра спроса: 0,33 и 0,66.
quantiles=shops_infa.quantile(q=[0.33,0.66])
quantiles

NameError: name 'shops_infa' is not defined

In [4]:
#"RClass" - функция для присвоения значения 1, 2 или 3  для значений, попадающих между соответствующими квантилями. 
#Эта функция для параметров спроса, в которых чем больше, тем лучше, а значит больше "очков" для каждого сегмента. 
#Это параметры: площадь магазина и средняя дневная выручка. Чем больше "очков", тем лучше спрос магазина.
def RClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.33]:
        return 1
    elif value <= quantiles_table[parameter_name][0.66]:
        return 2
    else:
        return 3
#"FMClass" - функция для присвоения значения 1, 2 или 3  для значений, попадающих между соответствующими квантилями. 
#Эта функция для параметров спроса, в которых чем меньше, тем лучше, а значит больше "очков" для каждого сегмента. 
#Это параметры: площадь магазина и средняя дневная выручка.    
def FMClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.33]:
        return 3   
    elif value <= quantiles_table[parameter_name][0.66]: 
        return 2
    else:
        return 1

In [5]:
    #Присвоение "очков" каждому магазину и его параметрам и получение общего рейтинга "Score".
shops_infa['square_0'] = shops_infa['square'].apply(RClass, args=('square',quantiles))

shops_infa['nearest_metro_distance_0'] = shops_infa['nearest_metro_distance'].apply(FMClass, args=('nearest_metro_distance',quantiles))

shops_infa['avg_daily_tt_revenue_0'] = shops_infa['avg_daily_tt_revenue'].apply(RClass, args=('avg_daily_tt_revenue',quantiles))

shops_infa['nearest_bus_stop_distance_0'] = shops_infa['nearest_bus_stop_distance'].apply(FMClass, args=('nearest_bus_stop_distance',quantiles))

shops_infa['Score'] = shops_infa['square_0'] + shops_infa['nearest_metro_distance_0']+\
                        shops_infa['avg_daily_tt_revenue_0'] + shops_infa['nearest_bus_stop_distance_0']
shops_infa

NameError: name 'shops_infa' is not defined

In [6]:
#определение уникальных значений рейтинга, которые получились. 
#Видно, что значения "3" нет, поэтому можно разбить на три равные группы.
shops_infa.Score.unique()

NameError: name 'shops_infa' is not defined

In [7]:
#Отбор магазинов с плохим рейтингом в сегмент "bad_location"
bad_location=shops_infa.query('Score==4 or Score==5 or Score==6')
bad_location

NameError: name 'shops_infa' is not defined

In [8]:
#Получение номеров магазинов, которые относятся к сегменту по спросу "bad location"
bad_location.id_tt.unique()

NameError: name 'bad_location' is not defined

In [9]:
#Подсчет числа магазинов из сегмента "bad location"
bad_location.id_tt.nunique()

NameError: name 'bad_location' is not defined

In [10]:
#Отбор магазинов со средним рейтингом в сегмент "good_location"
good_location=shops_infa.query('Score==7 or Score==8 or Score==9')
good_location

NameError: name 'shops_infa' is not defined

In [11]:
#Получение номеров магазинов, которые относятся к сегменту по спросу "good location"
good_location.id_tt.unique()

NameError: name 'good_location' is not defined

In [12]:
#Подсчет числа магазинов из сегмента "good location"
good_location.id_tt.nunique()

NameError: name 'good_location' is not defined

In [13]:
#Отбор магазинов с лучшим рейтингом в сегмент "the_best_location"
the_best_location=shops_infa.query('Score==10 or Score==11 or Score==12')
the_best_location

NameError: name 'shops_infa' is not defined

In [14]:
#Получение номеров магазинов, которые относятся к сегменту по спросу "the best location"
the_best_location.id_tt.unique()

NameError: name 'the_best_location' is not defined

In [15]:
#Подсчет числа магазинов из сегмента "the best location"
the_best_location.id_tt.nunique()

NameError: name 'the_best_location' is not defined

# 2). Нахождение магазинов с падающей выручкой относительно предыдущего месяца.

Получение данных о покупках за март, апрель и май.\
CheckId - идентификатор покупки \
id_tt_cl - код магазина\
id_tov_cl - код товара\
Price_retail - цена продукта, потупающего со склада\ 
Quantity - количество покупаемого товара\
BasePrice - цена продаваемого товара с учетом скидок\ 
BaseSum - стоимость товара с учетом скидок и \
date_ch - дата покупки\
time_ch - время покупки\
order_type - тип онлайн или оффлайн покупка

In [16]:
#считывание данных покупок за март, меняем тип данных колонки даты
march=pd.read_csv('1_march.csv', sep=';')
march.date_ch=pd.to_datetime(march.date_ch)
march.head(5)

FileNotFoundError: [Errno 2] No such file or directory: '1_march.csv'

In [17]:
#считывание данных покупок за апрель, меняем тип данных колонки даты
april=pd.read_csv('1_april.csv', sep=';')
april.date_ch=pd.to_datetime(april.date_ch)
april.head(5)

FileNotFoundError: [Errno 2] No such file or directory: '1_april.csv'

In [18]:
#считывание данных покупок за май, меняем тип данных колонки даты
may=pd.read_csv('1_may.csv', sep=';')
may.date_ch=pd.to_datetime(may.date_ch)
may.head(5)

FileNotFoundError: [Errno 2] No such file or directory: '1_may.csv'

In [19]:
#нахождение выручки магазина за каждый день в апреле
april_profit_12192=april.query('order_type=="offline" and id_tt_cl==12192')\
                        .groupby(['date_ch'],as_index=False)\
                        .agg({'BaseSum':'sum'})
april_profit_12192.head(5)

NameError: name 'april' is not defined

In [20]:
#загрузка библиотеки для построения графика
import seaborn as sns
#установка параметров графика
sns.set(rc={'figure.figsize':(20,7)}, style="whitegrid")
# построение графика получаемой выручки по дням для одного из магазинов в апреле
sns.lineplot(x='date_ch',y='BaseSum',data=april_profit_12192)

NameError: name 'april_profit_12192' is not defined

Как в одном магазине, так и во всех видим падение выручки в выходные дни. Работники это связывают с тем, что люди уезжают на выходные за город, на дачу и т.д.

In [21]:
#нахождение выручки за каждый из месяцев и каждый из дней
march_profit=march.query('order_type=="offline"')\
                  .groupby(['id_tt_cl','date_ch'],as_index=False)\
                  .agg({'BaseSum':'sum'})
march_profit.head(5)

NameError: name 'march' is not defined

In [22]:
april_profit=april.query('order_type=="offline"')\
                  .groupby(['id_tt_cl','date_ch'],as_index=False)\
                  .agg({'BaseSum':'sum'})
april_profit.head(5)

NameError: name 'april' is not defined

In [23]:
may_profit=may.query('order_type=="offline"')\
              .groupby(['id_tt_cl','date_ch'],as_index=False)\
              .agg({'BaseSum':'sum'})
may_profit.head(5)

NameError: name 'may' is not defined

In [24]:
#объединение выручки по дням в один датафрейм, состоящий из 3х месяцев
april_march_may_profit=pd.concat([march_profit,april_profit,may_profit],axis=0)
april_march_may_profit.head(5)

NameError: name 'march_profit' is not defined

In [25]:
# построение графика получаемой выручки по дням для одного из магазинов за три месяца
sns.lineplot(x='date_ch',y='BaseSum',data=april_march_may_profit)

NameError: name 'april_march_may_profit' is not defined

По графику заметны провалы выручки 24 апреля, 1 мая, 8 мая. Все это праздники, совпадающие с выходными днями. 24 апреля - Пасха, 1 мая - День труда, 9 мая - День Победы

In [26]:
#Нахождение выручки за март
total_march_profit=march_profit.groupby('id_tt_cl',as_index=False)\
                               .agg({'BaseSum':'sum'})
total_march_profit.head(5)

NameError: name 'march_profit' is not defined

In [27]:
#Нахождение выручки за апрель
total_april_profit=april_profit.groupby('id_tt_cl',as_index=False)\
                               .agg({'BaseSum':'sum'})
total_april_profit.head(5)

NameError: name 'april_profit' is not defined

In [28]:
#Нахождение выручки за май
total_may_profit=may_profit.groupby('id_tt_cl',as_index=False)\
                           .agg({'BaseSum':'sum'})
total_may_profit.head(5)

NameError: name 'may_profit' is not defined

In [29]:
#датафрейм с выручки за март и апрель
total_march_april=total_march_profit.merge(total_april_profit, on='id_tt_cl')
total_march_april.rename(columns = {'BaseSum_x' : 'March', 'BaseSum_y' : 'April'}, inplace = True)
total_march_april.head(5)


NameError: name 'total_march_profit' is not defined

In [30]:
#датафрейм с выручки за март, апрель, май
total_march_april_may=total_march_april.merge(total_may_profit, on='id_tt_cl')
total_march_april_may.rename(columns = {'BaseSum' : 'May'}, inplace = True)
total_march_april_may.head(5)

NameError: name 'total_march_april' is not defined

In [31]:
#формирование датафрейма с изменением процента выручки относительно предыдущего месяца
difference = pd.DataFrame(columns=['Shop','April-March', 'May-April'])
difference['April-March']=total_march_april_may.April/\
                          total_march_april_may.March*100-100
difference['May-April']=total_march_april_may.May/\
                          total_march_april_may.April*100-100
difference['Shop']=total_march_april_may.id_tt_cl
difference.head(5)


NameError: name 'total_march_april_may' is not defined

Находим недельные выручки по магазинам

In [32]:
#формирование столбца с номерами недель для датасета с выручкой за день для каждого магазина
Week=april_march_may_profit.date_ch.dt.strftime('%W')
Week.head(5)

NameError: name 'april_march_may_profit' is not defined

In [33]:
#создание датафрейма с выручкой за каждый день каждого магазина с обозначением номера недели вместо даты
Weak_MAM=pd.DataFrame({'id_tt_cl':april_march_may_profit.id_tt_cl\
                       ,'Week':Week\
                       ,'BaseSum':april_march_may_profit.BaseSum})
Weak_MAM.head(5)

NameError: name 'april_march_may_profit' is not defined

In [34]:
#убираем из датафрейма данные о последних неделях, так как там неполные недели, что даст неверные данные для выручки за неделю
Weak_MAM_without_09_22=Weak_MAM.query("Week!='22' and Week!='09'")
Weak_MAM_without_09_22.head(5)

NameError: name 'Weak_MAM' is not defined

In [35]:
#формирование датасета с выручкой за каждую неделю каждого магазина
profit_for_every_weak_MAM=Weak_MAM_without_09_22.groupby(['id_tt_cl','Week']).agg({'BaseSum':'sum'})
profit_for_every_weak_MAM.head(5)

NameError: name 'Weak_MAM_without_09_22' is not defined

In [36]:
#формирование таблицы pivot для выручки с номером недели для колонок и номером магазина в строках
pivot_weeks=pd.pivot_table(profit_for_every_weak_MAM,index='id_tt_cl', columns='Week',values='BaseSum')
pivot_weeks.head(5)

NameError: name 'profit_for_every_weak_MAM' is not defined

In [37]:
# убираем данные по магазинам 16250 и 11217, так как в них отсттствует около половины данных
pivot_weeks_witjout_NaN=pivot_weeks.query("id_tt_cl!='16250' and id_tt_cl!='11217'")
pivot_weeks_witjout_NaN.head(5)

NameError: name 'pivot_weeks' is not defined

In [38]:
#получение разницы в приыбыли относительно предыдущей недели для каждого магазина
Every_week_diff=pivot_weeks_witjout_NaN.pct_change(axis='columns')*100
Every_week_diff=Every_week_diff.round(2)
Every_week_diff.head(5)

NameError: name 'pivot_weeks_witjout_NaN' is not defined

присваиваем нетоповым магазам (99% от всех) прибыль топовых


In [39]:
#суммарная выручка для двух последних недель для всех 82х магазинов
two_weeks_old_profit_of_all_location=pivot_weeks_witjout_NaN[['20','21']].sum()
TOTAL_two_weeks_old_profit_of_all_location=two_weeks_old_profit_of_all_location.sum()
TOTAL_two_weeks_old_profit_of_all_location

NameError: name 'pivot_weeks_witjout_NaN' is not defined

In [40]:
#Находим топовые магазины в каждом сегменте 
#потому что даже в сегменте магазинов с отличной локацией может быть отрийцательная прибыль
#best_location
best_location_difference=difference.query('Shop==10778 or Shop==10786 or Shop==11020 or Shop==11634 or Shop==11650 or Shop==\
        11657 or Shop==11783 or Shop== 11847 or Shop==11955\
        or Shop==12084 or Shop== 12132 or Shop== 12192 or Shop== 12392 or Shop== 12860 or Shop==\
        13018 or Shop==14025 or Shop==14030 or Shop==14396\
        or Shop==14855 or Shop== 15986')
best_location_difference
#good_location
good_location_difference=difference.query('Shop==10780 or Shop==10904 or Shop==11103 \
        or Shop==11159 or Shop==11290 or   Shop==11309    or Shop==11420 or Shop== 11458 or Shop==11610  or  \
         Shop==11662 or Shop== 11673 or Shop== 11697 or Shop== 11750 or \
         Shop== 11766 or Shop==11777 or Shop==11841 or Shop==11888 or Shop==11901 or Shop==11937 or Shop== 11958 or\
         Shop==12025 or Shop== 12299 or Shop== 12326 or \
         Shop== 12338 or Shop==12481 or Shop== 12596 or Shop== 12834 or Shop== 12871 or\
         Shop==12879 or Shop== 12892 or Shop== 13262 or Shop== 13316 or  \
         Shop==13342 or Shop== 13569 or Shop== 14228 or Shop== 14899 or  \
          Shop==15175 or Shop== 15239 or Shop== 15753 or Shop== 15970 or Shop== 16130')
good_location_difference
#bad_location
bad_location_difference=difference.query('Shop==11658 or Shop==11723 or Shop==11855 \
        or Shop==11980 or Shop==12004 or   Shop==12072    or Shop==12261 or Shop== 12285 or Shop==12332  or  \
         Shop==12335 or Shop== 12458 or Shop== 12518 or Shop== 12712 or \
         Shop== 12821 or Shop==12895 or Shop==12938 or Shop==13012 or Shop==13168 or Shop==13429 or Shop== 13623 or\
         Shop==13767 or Shop== 13926 or Shop== 15834')
bad_location_difference

,Shop,April-March,May-April


In [41]:
pivot_weeks_witjout_NaN


NameError: name 'pivot_weeks_witjout_NaN' is not defined

In [42]:
pivot_weeks_witjout_NaN=pivot_weeks_witjout_NaN.reset_index().rename(columns={'id_tt_cl':'Shop'})

NameError: name 'pivot_weeks_witjout_NaN' is not defined

In [43]:
#Находим топовые магазины в каждом сегменте
#потому что даже в сегменте магазинов с отличной локацией может быть отрийцательная прибыль
#best_location
old_profit_of_best_location=pivot_weeks_witjout_NaN.query('Shop==10778 or Shop==10786 or Shop==11020 or Shop==11634 or Shop==11650 or Shop==\
        11657 or Shop==11783 or Shop== 11847 or Shop==11955\
        or Shop==12084 or Shop== 12132 or Shop== 12192 or Shop== 12392 or Shop== 12860 or Shop==\
        13018 or Shop==14025 or Shop==14030 or Shop==14396\
        or Shop==14855 or Shop== 15986')
old_profit_of_best_location.shape


NameError: name 'pivot_weeks_witjout_NaN' is not defined

In [44]:
#good_location
old_profit_of_good_location=pivot_weeks_witjout_NaN.query('Shop==10780 or Shop==10904 or Shop==11103 \
        or Shop==11159 or Shop==11290 or   Shop==11309    or Shop==11420 or Shop== 11458 or Shop==11610  or  \
         Shop==11662 or Shop== 11673 or Shop== 11697 or Shop== 11750 or \
         Shop== 11766 or Shop==11777 or Shop==11841 or Shop==11888 or Shop==11901 or Shop==11937 or Shop== 11958 or\
         Shop==12025 or Shop== 12299 or Shop== 12326 or \
         Shop== 12338 or Shop==12481 or Shop== 12596 or Shop== 12834 or Shop== 12871 or\
         Shop==12879 or Shop== 12892 or Shop== 13262 or Shop== 13316 or  \
         Shop==13342 or Shop== 13569 or Shop== 14228 or Shop== 14899 or  \
          Shop==15175 or Shop== 15239 or Shop== 15753 or Shop== 15970 or Shop== 16130')
old_profit_of_good_location.shape

NameError: name 'pivot_weeks_witjout_NaN' is not defined

In [45]:
#bad_location
old_profit_of_bad_location=pivot_weeks_witjout_NaN.query('Shop==11658 or Shop==11723 or Shop==11855 \
        or Shop==11980 or Shop==12004 or   Shop==12072    or Shop==12261 or Shop== 12285 or Shop==12332  or  \
         Shop==12335 or Shop== 12458 or Shop== 12518 or Shop== 12712 or \
         Shop== 12821 or Shop==12895 or Shop==12938 or Shop==13012 or Shop==13168 or Shop==13429 or Shop== 13623 or\
         Shop==13767 or Shop== 13926 or Shop== 15834')
old_profit_of_bad_location.shape

NameError: name 'pivot_weeks_witjout_NaN' is not defined

Находим топовый магаз из каждого сегмента (локации) и находим выручку за две недели. Запоминаем эту выручку и присваеваем ее всем магазинам сегмента. Для best_location - это 20 магазинов, для good_location - 41, а для bad_location 23.

In [46]:
#вырчука теста (гипотетическая) для двух недельного теста сегмента best_location
old_profit_of_best_location_11783=old_profit_of_best_location.query('Shop==11783')
two_weeks_test_profit_of_best_location=(old_profit_of_best_location_11783['20']+old_profit_of_best_location_11783['21'])
two_weeks_test_profit_of_best_location

NameError: name 'old_profit_of_best_location' is not defined

In [47]:
#вырчука теста (гипотетическая) для двух недельного теста сегмента good_location
old_profit_of_good_location_11783=old_profit_of_good_location.query('Shop==15175')
two_weeks_test_profit_of_good_location=(old_profit_of_good_location_11783['20']+old_profit_of_good_location_11783['21'])
two_weeks_test_profit_of_good_location

NameError: name 'old_profit_of_good_location' is not defined

In [48]:
#вырчука теста (гипотетическая) для двух недельного теста сегмента bad_location
old_profit_of_bad_location_11783=old_profit_of_bad_location.query('Shop==11855')
two_weeks_test_profit_of_bad_location=(old_profit_of_bad_location_11783['20']+old_profit_of_bad_location_11783['21'])
two_weeks_test_profit_of_bad_location

NameError: name 'old_profit_of_bad_location' is not defined

Получение выручки для трех магазинов всех трех сегментов из категории "топовые", тоесть у которых выручка за последние два месяца росла сильнее всех. Числа 20, 41, 23 - то количество магазинов в каждом сегменте

In [49]:
TOTAL_two_weeks_test_profit_of_all_location=two_weeks_test_profit_of_best_location[23]*20\
                                            +two_weeks_test_profit_of_good_location[77]*41\
                                            +two_weeks_test_profit_of_bad_location[26]*23
                                            
TOTAL_two_weeks_test_profit_of_all_location

NameError: name 'two_weeks_test_profit_of_best_location' is not defined

In [50]:
#Делим выручку после гипотетического теста с присвоением 

In [51]:
#получили разницу в  процентов увеличения прибыли за счет приравнивания всех магазинов под один
#за две недели
TOTAL_two_weeks_test_profit_of_all_location/TOTAL_two_weeks_old_profit_of_all_location

NameError: name 'TOTAL_two_weeks_test_profit_of_all_location' is not defined

# 3). Выбор магазинов в каждом сегменте по ассортименту

Цель: отобрать из каждого сегмента магазины, которые максимально похожи по количеству уникальных товаров (по ассортименту) на магазин с растущей выручкой.
Обозначения:\
id_tt - номер магазина \
id_tov - номер товара\
name_tov - название  товара\
Вначале определяется количество уникальных товаров в магазине с растущей выручкой. Затем с помощью алгоритма вычисляется для каждого магазина из каждого сегмента процент, на который отличается количество уникальных товаров в сравниваемых магазинах. Если различий меньше, чем 3 % (подобрано эмпирически), то магазины считаются похожими по ассортименты и подходящими для А/В теста.

In [52]:
#считываем данные по товарам
ost=pd.read_csv('1_ost.csv', sep=';')
ost.head(5)

FileNotFoundError: [Errno 2] No such file or directory: '1_ost.csv'

In [53]:
#Магазин с растущей выручкой за два месяца  для сравнения в сегменте best_location
best_11783_top=ost.query('id_tt==11783').groupby('name_tov').agg({'id_tov':'count'})
best_11783_top

NameError: name 'ost' is not defined

In [54]:
#Перечисление номеров магазинов из сегмента best_location
best=(10778, 10786, 11020, 11634, 11650, 11657, 11783, 11847, 11955,
       12084, 12132, 12192, 12392, 12860, 13018, 14025, 14030, 14396,
       14855, 15986)


In [55]:
#для сегмента best_Location алгоритм, выдающий количество процентов, на которое отличается ассортимент в каждом магазине от
#ассортимента магазина с растущей выручкой с количеством уникальных товаров 4085.
inputt=pd.DataFrame()
output=pd.DataFrame(columns = {"id_tt" ,"count" })
for i in best:
    
        inputt=ost.query('id_tt == @i').groupby('name_tov').agg({'id_tov':'count'})
        rows_columns=inputt.shape
        rows=rows_columns[0]
        h=rows/4085*100-100
        if abs(h)<3:
            output=output.append({'id_tt':i, 'count':h},ignore_index=True)
    
    
print(output)

NameError: name 'ost' is not defined

In [56]:
#Магазин с растущей выручкой за два месяца  для сравнения в сегменте good_location
good_15753_top=ost.query('id_tt==15753').groupby('name_tov').agg({'id_tov':'count'})
good_15753_top

NameError: name 'ost' is not defined

In [57]:
#Перечисление номеров магазинов из сегмента good_location
good=(10780, 11103, 11159, 11290, 11458, 11610, 11662,\
11673, 11697, 11750, 11766, 11777, 11841, 11888, 11901, 11937,\
11958, 12326, 12338, 12481, 12834, 12871, 12892, 13262, 13316, 13342,\
13569, 14899, 15175, 15239, 15970,\
10904, 11309, 11420,  12025, 12879, 11610, 12326, 11662, 11673, 13342)

In [58]:
#для сегмента good_Location алгоритм, выдающий количество процентов, на которое отличается ассортимент в каждом магазине от
#ассортимента магазина с растущей выручкой с количеством уникальных товаров 4094.
inputt=pd.DataFrame()
output=pd.DataFrame(columns = {"id_tt" ,"count" })
for i in good:
    
        inputt=ost.query('id_tt == @i').groupby('name_tov').agg({'id_tov':'count'})
        rows_columns=inputt.shape
        rows=rows_columns[0]
        h=rows/4094*100-100
        if abs(h)<3:
            output=output.append({'id_tt':i, 'count':h},ignore_index=True)
    
print(output) 

NameError: name 'ost' is not defined

In [59]:
#Магазин с растущей выручкой за два месяца для сравнения в сегменте bad_location
bad_11855_top=ost.query('id_tt==11855').groupby('name_tov').agg({'id_tov':'count'})
bad_11855_top.shape

NameError: name 'ost' is not defined

In [60]:
#Перечисление номеров магазинов из сегмента bad_location
bad=(11658, 11723, 11855, 11980, 12004, 12072, 12261, 12285, 12332,
       12335, 12458, 12518, 12712, 12821, 12895, 12938, 13012, 13168,
       13429, 13623, 13767, 13926, 15834)

In [61]:
#для сегмента bad_Location алгоритм, выдающий количество процентов, на которое отличается ассортимент в каждом магазине от
#ассортимента магазина с растущей выручкой с количеством уникальных товаров 4045.
inputt=pd.DataFrame()
output=pd.DataFrame(columns = {"id_tt" ,"count" })
for i in bad:
    
        inputt=ost.query('id_tt == @i').groupby('name_tov').agg({'id_tov':'count'})
        rows_columns=inputt.shape
        rows=rows_columns[0]
        h=rows/4045*100-100
        if abs(h)<3:
            output=output.append({'id_tt':i, 'count':h},ignore_index=True)
    
print(output)

NameError: name 'ost' is not defined

# 4). Подготовка к A/B тесту

In [62]:
#Переименование колонок, для удобства.
weekly_diff_profit_all_shops=Every_week_diff.rename(columns={'20':'prelast','21':'last'})

NameError: name 'Every_week_diff' is not defined

In [63]:
#Вычисление процента изменения выручки в целом за две недели по данным за последние две недели.
weekly_diff_profit_all_shops['two_last_week']=\
    (weekly_diff_profit_all_shops['prelast']/100+1)*\
    (weekly_diff_profit_all_shops['last']/100+1)
weekly_diff_profit_all_shops['two_last_week_percent']=\
    (weekly_diff_profit_all_shops['two_last_week']-1)*100
weekly_diff_profit_all_shops
#Получили изменение выручки по всем магазинам за последние две недели

NameError: name 'weekly_diff_profit_all_shops' is not defined

In [64]:
#отбор магазинов в контрольную группу по совпадению ассортимента не менее 97%.
Top_weekly_shops_control=weekly_diff_profit_all_shops.query('id_tt_cl==11657 or id_tt_cl==11159  or id_tt_cl==12261\
or id_tt_cl==10786 or id_tt_cl==11847 or id_tt_cl==11855 or id_tt_cl==12084 or id_tt_cl==12192\
or id_tt_cl==14228 or id_tt_cl==15753 or id_tt_cl==16130\
or id_tt_cl==11766 or id_tt_cl==12299 or id_tt_cl==12596\
or id_tt_cl==12834 or id_tt_cl==14899 or id_tt_cl==15175\
or id_tt_cl==12879 or id_tt_cl==11783 or id_tt_cl==10780')
Top_weekly_shops_control=Top_weekly_shops_control.reset_index()
Top_weekly_shops_control

NameError: name 'weekly_diff_profit_all_shops' is not defined

In [65]:
#отбор магазинов в тестовую группу по совпадению ассортимента не менее 97%.
Not_top_weekly_shops_test=weekly_diff_profit_all_shops.query('id_tt_cl==10904 or id_tt_cl==11309  or id_tt_cl==11420\
or id_tt_cl==12025\
or id_tt_cl==12879 or id_tt_cl==11610 or id_tt_cl==12326\
or id_tt_cl==11662 or id_tt_cl==11673 or id_tt_cl==13342')
Not_top_weekly_shops_test=Not_top_weekly_shops_test.reset_index()
Not_top_weekly_shops_test


NameError: name 'weekly_diff_profit_all_shops' is not defined

In [66]:
#получение данных для A/B теста с 10 магазинами в тестовой группе, где будут изменения.
#В столбике test находятся значения, показывающие насколько изменилось в относительных единицах значение выручки за 2 недели
A_B_test=pd.DataFrame({'id_tt_cl':Not_top_weekly_shops_test.id_tt_cl,'test':Not_top_weekly_shops_test.two_last_week})
A_B_test

NameError: name 'Not_top_weekly_shops_test' is not defined

In [67]:
#получение данных для A/B теста с 20 магазинами в контрольной группе, где будут изменения.
#В столбике control находятся значения, показывающие насколько изменилось в относительных единицах значение выручки за 2 недели
A_B_control=pd.DataFrame({'id_tt_cl':Top_weekly_shops_control.id_tt_cl,'control':Top_weekly_shops_control.two_last_week})
A_B_control

NameError: name 'Top_weekly_shops_control' is not defined

Для А/B теста необходимо подготовить данные. Для использования критерия t-Стьюдента неодходимо соблюдения двух условий: гомогенность дисперсий и нормальность распределения средних значений выборок. 

In [68]:
#Условие гомогенности дисперсий выполняется
from scipy.stats import levene
levene(A_B_control.control, A_B_test.test)

NameError: name 'A_B_control' is not defined

In [69]:
#по критерию Шапиро-Уилка распределение контрольной группы нормально
import random
import scipy.stats as ss
sample=random.sample(list(A_B_control.control), 15)
ss.shapiro(sample)

NameError: name 'A_B_control' is not defined

In [70]:
#по normaltest распределение контрольной группы нормально
ss.normaltest(A_B_control.control)

NameError: name 'A_B_control' is not defined

In [71]:
#по normaltest распределение тестовой группы нормально
ss.normaltest(A_B_test.test)

NameError: name 'A_B_test' is not defined

In [72]:
#по критерию Шапиро-Уилка распределение тестовой группы нормально
import random
import scipy.stats as ss
sample=random.sample(list(A_B_test.test), 9)
ss.shapiro(sample)

NameError: name 'A_B_test' is not defined


Вывод:\                                                                                               \
1). Выдвинута гипотеза и подготовлены номера магазинов, которые должны использоваться в А/В тесте.\
2). Группа тестовая и группа контрольная имеют нормальное распределение и 
они имеют одинаковые дисперсии, а значит можно использовать     t-тест для анализа результатов А/В теста.
 